In [1]:
from setup_general import *

# xg boost


In [11]:
data = train_prep.copy()
rebalancing = False

features = data.drop('type', axis=1)
labels = data.type
# at least xgboost cannot deal with string labels
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(labels)
labels = label_encoder.transform(labels)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=0)

# possible rebalancing

In [ ]:
if rebalancing == True:
    from imblearn.over_sampling import SMOTE, ADASYN
    from collections import Counter

    def strat(data):
        b = Counter(data).values()
        a = Counter(data).keys()
        a = list(a)
        b = list(b)

        for i in range(len(a)):
            if b[i] < 100 :
                b[i] = 100
        return dict(zip(a, b))

    X_train, y_train = SMOTE(sampling_strategy=strat(y_train), random_state=1).fit_resample(X_train, y_train)

# Classifier

In [ ]:
bst = XGBClassifier(random_state=0)
# fit model
bst.fit(X_train, y_train)
# make predictions
preds = bst.predict(X_test)
val_acc = accuracy_score(y_test, preds)
val_acc

In [13]:
bst.save_model('models/xg/xg_model.json')

# submission from model

In [86]:
test_set = test_prep.drop('type', axis=1)
results = bst.predict(test_set)
submission = pd.DataFrame({'id': test_set.index ,'type': bst.predict(test_set)})
submission = submission.replace(type_lookup.id.to_list(), type_lookup.estonian.to_list())
submission.to_csv('data/submission_xgboost.csv', index=False)